In [ ]:
import geopandas as gpd
import pandas as pd
from sklearn import preprocessing

from core.gw import BandwidthSearch
from core.gw.ensemble import GWGradientBoostingClassifier, GWRandomForestClassifier
from core.gw.linear_model import GWLogisticRegression

In [ ]:
from glob import glob

glob("/data/uscuni-restricted/04_spatial_census/*")

In [ ]:
census = gpd.read_parquet(
    "/data/uscuni-restricted/04_spatial_census/nadzsjd_education_2021.parquet"
)

In [ ]:
total_pop = pd.read_csv(
    "/data/uscuni-restricted/04_spatial_census/total.csv",
    index_col=0,
    dtype={"nadzsjd": str},
)

In [ ]:
census = census.merge(total_pop, on="nadzsjd")

In [ ]:
variables = census.columns[census.columns.str.startswith("Obyvatelstvo -")]

In [ ]:
census[variables] = census[variables].div(census["Obyvatelstvo celkem"], axis=0)

In [ ]:
non_na = census.dropna(subset=variables)

In [ ]:
non_na.shape

In [ ]:
clusters = pd.read_csv(
    "/data/uscuni-restricted/geometries/cluster_assignment_v3.csv",
    dtype={"kod_nadzsj_d": str},
)
cluster_mapping = pd.read_parquet(
    "/data/uscuni-ulce/processed_data/clusters/cluster_mapping_v3.pq"
)

In [ ]:
cluster_mapping.loc[8]

In [ ]:
non_na = non_na.merge(clusters, left_on="nadzsjd", right_on="kod_nadzsj_d")

In [ ]:
search = BandwidthSearch(
    GWRandomForestClassifier,
    fixed=False,
    n_jobs=-1,
    search_method="golden_section",
    criterion="aicc",
    max_bandwidth=1000,
    max_iterations=10,
    tolerance=0.05,
    verbose=True,
    batch_size=500,
    min_proportion=0.05,
    class_weight="balanced",
)
search.fit(
    non_na[variables],
    non_na["final_without_noise"].map(cluster_mapping[3]) == 1,
    non_na.representative_point(),
)

In [ ]:
search.oob_scores.sort_index().plot()

In [ ]:
search.optimal_bandwidth

In [ ]:
rf = GWRandomForestClassifier(
    bandwidth=750,
    fixed=False,
    n_jobs=-1,
    keep_models=False,
    # temp_folder="/tmp",
    batch_size=1000,
    min_proportion=0.05,
    class_weight="balanced",
)
rf.fit(
    non_na[variables],
    non_na["final_without_noise"].map(cluster_mapping[4]) == 2,
    non_na.representative_point(),
)

In [ ]:
rf.oob_score_

In [ ]:
rf.score_

In [ ]:
non_na.plot(
    rf.local_oob_score_,
    legend=True,
    figsize=(16, 8),
    missing_kwds=dict(color="lightgray"),
).set_axis_off()

In [ ]:
non_na.plot(
    rf.focal_proba_[True],
    legend=True,
    figsize=(16, 8),
    missing_kwds=dict(color="lightgray"),
).set_axis_off()

In [ ]:
non_na.plot(
    rf.feature_importances_[
        "Obyvatelstvo - věk: 15 a více - nejvyšší dosažené vzdělání:  vysokoškolské - muži"
    ],
    legend=True,
    figsize=(16, 8),
    missing_kwds=dict(color="lightgray"),
).set_axis_off()

In [ ]:
focal_pred = rf.focal_proba_.iloc[:, 0] > 0.5

In [ ]:
from sklearn import metrics

y = non_na["final_without_noise"].map(cluster_mapping[4]) == 2

In [ ]:
metrics.accuracy_score(y, focal_pred)

In [ ]:
metrics.f1_score(y, focal_pred, average="macro")

In [ ]:
metrics.f1_score(y, focal_pred, average="micro")

In [ ]:
metrics.f1_score(y, focal_pred, average="weighted")

In [ ]:
rf.oob_score_